In [1]:
import collections
import hashlib
import random
import binascii
import sys
from Crypto.Cipher import AES





In [0]:
import Padding

In [2]:
def enc_long(n):
    '''Encodes arbitrarily large number n to a sequence of bytes.
    Big endian byte order is used.'''
    s = ""
    while n > 0:
        s = chr(n & 0xFF) + s
        n >>= 8
    return s

In [3]:
BLOCK_SIZE = 16  # Bytes
pad = lambda s: s + (BLOCK_SIZE - len(s) % BLOCK_SIZE) * \
                chr(BLOCK_SIZE - len(s) % BLOCK_SIZE)
unpad = lambda s: s[:-ord(s[len(s) - 1:])]

In [4]:
def encrypt(plaintext,key, mode):
	encobj = AES.new(key,mode)
	return(encobj.encrypt(plaintext))


In [5]:
def decrypt(ciphertext,key, mode):
	encobj = AES.new(key,mode)
	return(encobj.decrypt(ciphertext))

In [6]:
EllipticCurve = collections.namedtuple('EllipticCurve', 'name p a b g n h')

In [7]:
curve = EllipticCurve(
    'secp256k1',
    # Field characteristic.
    p=0xfffffffffffffffffffffffffffffffffffffffffffffffffffffffefffffc2f,
    # Curve coefficients.
    a=0,
    b=7,
    # Base point.
    g=(0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
       0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8),
    # Subgroup order.
    n=0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141,
    # Subgroup cofactor.
    h=1,
)

In [8]:
def inverse_mod(k, p):
    """Returns the inverse of k modulo p.
    This function returns the only integer x such that (x * k) % p == 1.
    k must be non-zero and p must be a prime.
    """
    if k == 0:
        raise ZeroDivisionError('division by zero')

    if k < 0:
        # k ** -1 = p - (-k) ** -1  (mod p)
        return p - inverse_mod(-k, p)

    # Extended Euclidean algorithm.
    s, old_s = 0, 1
    t, old_t = 1, 0
    r, old_r = p, k

    while r != 0:
        quotient = old_r // r
        old_r, r = r, old_r - quotient * r
        old_s, s = s, old_s - quotient * s
        old_t, t = t, old_t - quotient * t

    gcd, x, y = old_r, old_s, old_t

    assert gcd == 1
    assert (k * x) % p == 1

    return x % p

In [9]:
def is_on_curve(point):
    """Returns True if the given point lies on the elliptic curve."""
    if point is None:
        # None represents the point at infinity.
        return True

    x, y = point

    return (y * y - x * x * x - curve.a * x - curve.b) % curve.p == 0

In [10]:
def point_neg(point):
    """Returns -point."""
    assert is_on_curve(point)

    if point is None:
        # -0 = 0
        return None

    x, y = point
    result = (x, -y % curve.p)

    assert is_on_curve(result)

    return result

In [11]:
def point_add(point1, point2):
    """Returns the result of point1 + point2 according to the group law."""
    assert is_on_curve(point1)
    assert is_on_curve(point2)

    if point1 is None:
        # 0 + point2 = point2
        return point2
    if point2 is None:
        # point1 + 0 = point1
        return point1

    x1, y1 = point1
    x2, y2 = point2

    if x1 == x2 and y1 != y2:
        # point1 + (-point1) = 0
        return None

    if x1 == x2:
        # This is the case point1 == point2.
        m = (3 * x1 * x1 + curve.a) * inverse_mod(2 * y1, curve.p)
    else:
        # This is the case point1 != point2.
        m = (y1 - y2) * inverse_mod(x1 - x2, curve.p)

    x3 = m * m - x1 - x2
    y3 = y1 + m * (x3 - x1)
    result = (x3 % curve.p,
              -y3 % curve.p)

    assert is_on_curve(result)

    return result

In [12]:
def scalar_mult(k, point):
    """Returns k * point computed using the double and point_add algorithm."""
    assert is_on_curve(point)

    if k % curve.n == 0 or point is None:
        return None

    if k < 0:
        # k * point = -k * (-point)
        return scalar_mult(-k, point_neg(point))

    result = None
    addend = point

    while k:
        if k & 1:
            # Add.
            result = point_add(result, addend)

        # Double.
        addend = point_add(addend, addend)

        k >>= 1

    assert is_on_curve(result)

    return result

In [13]:
def make_keypair():
    """Generates a random private-public key pair."""
    private_key = random.randrange(1, curve.n)
    public_key = scalar_mult(private_key, curve.g)

    return private_key, public_key

In [14]:
def verify_signature(public_key, message, signature):
    z = hash_message(message)

    r, s = signature

    w = inverse_mod(s, curve.n)
    u1 = (z * w) % curve.n
    u2 = (r * w) % curve.n

    x, y = point_add(scalar_mult(u1, curve.g),
                     scalar_mult(u2, public_key))

    if (r % curve.n) == (x % curve.n):
        return 'signature matches'
    else:
        return 'invalid signature'

In [15]:
message="Hello"
if (len(sys.argv)>1):
	message=str(sys.argv[1])

In [16]:
dA, Qa = make_keypair()
print("Private key:", hex(dA))
print(("Public key: (0x{:x}, 0x{:x})".format(*Qa)))

Private key: 0xe620f530a96e3a47cf0ad58beb99a9aaf0414e58ccec5cf71f449592128c52e
Public key: (0xf39d58fd3d7c47c0b02104d43cd17a355e0ca2fb8418d6c6a14b09889609676d, 0xc9ad199c11da1bafe7d6ffe376ac768702130db415e5da7097e416abc0c3d375)


In [19]:
r = random.randint(0, 2**128)

rG = scalar_mult(r,curve.g)
S = scalar_mult(r,Qa)
print("Random value: " , r)
print("rG: " , rG)
print("\n\n======Symmetric key========")
print("Encryption key:",S[0],str(S[0]))
password='hello'

Random value:  125945476372480411682012657926855769563
rG:  (77727987616921594621675615650746644005351131759422055612764749389235593014283, 19190430888330267798434745443781823633675860626003424121219317121613703096324)


======Symmetric key========
Encryption key: 16956040424460820237248880336758516766703512530337594922588882282019587240627 16956040424460820237248880336758516766703512530337594922588882282019587240627


In [0]:
key = hashlib.sha256(str(S[0]).encode()).digest()

message = Padding.appendPadding(message,blocksize=Padding.AES_blocksize,mode=0)

ciphertext = encrypt(message.encode(),key,AES.MODE_ECB)




print("Encrypted:\t",binascii.hexlify(ciphertext))

In [0]:
Snew = scalar_mult(dA,rG)
key = hashlib.sha256(str(Snew[0]).encode()).digest()

text = decrypt(ciphertext,key,AES.MODE_ECB)


print("Decrypted:\t",Padding.removePadding(text.decode(),mode=0))